In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
path = 'gdrive/MyDrive/pytorch_geo_gcn/'
path = ''

In [7]:
# !pip install torch==version
TORCH_version = torch.__version__
# TORCH = format_pytorch_version(TORCH_version)
torch.__version__

'1.8.1+cu101'

In [ ]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

print(torch.version.cuda,TORCH)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 

10.1 1.8.1
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.1+cu101.html


Defining the GCN class/layer in the following cell


In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from  torch_geometric.nn import GCNConv, Sequential



class GCN(nn.Module):
    def __init__(self, num_feat, num_hid, num_class,dropout):

        super(GCN, self).__init__()
        self.gc1 = GCNConv(num_feat, num_hid)
        self.gc2 = GCNConv(num_hid, num_hid)
        # self.gc3 = GCNConv(num_hid, num_hid)
        self.gc4 = GCNConv(num_hid, num_class)
        # self.gc5 = GCNConv(num_hid, num_class)
        self.dropout = dropout

    def forward(self,x,edge_index,edge_weight): 
        x= F.relu(self.gc1(x=x, edge_index= edge_index , edge_weight = edge_weight))
        x = F.dropout(x, self.dropout, training=self.training)
        x = torch.sigmoid(self.gc2(x=x.float(),edge_index= edge_index, edge_weight= edge_weight))
        # x = F.dropout(x, self.dropout, training=self.training)
        # x = torch.sigmoid(self.gc3(x=x.float(),edge_index= edge_index, edge_weight= edge_weight))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc4(x=x.float(),edge_index= edge_index, edge_weight= edge_weight)
        return F.log_softmax(x, dim=1)
    
    

The next cell contains some utility functions

In [7]:
import numpy as np
import torch
import scipy.sparse as sp
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from matplotlib import pyplot
from scipy import stats
import os
import time
from sklearn import preprocessing, feature_extraction, model_selection
from copy import deepcopy
from IPython.display import display, HTML
import scipy.sparse
from scipy.spatial.distance import cdist
from scipy.spatial import KDTree
import networkx as nx
from torch import LongTensor
from torch import int64
from torch import tensor



def get_labels(labels):
    classes = set(labels)
    classes_dict = {c: i for i,c in enumerate(classes)}
    labels = np.array([classes_dict[i] for i in labels])
    return labels



def normalize(mx):
    """Row-normalize matrix - Unchanged from Kipfs"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx


def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


def distance_matrix(cutoff, points_1, points_2=None, output='dict'):
    
    """
    This function computes a distance matrix between points in 'points_1' and 'points_2'
    that are within 'cutoff' of each other.
    
    ***Input
    cutoff   : the distance cutoff
    points_1 : a numpy array of coordinates
    points_2 : a numpy array of coordinates
    output   : form of output.
    ***Output
            'dict': (default) is a dictionary with every pair of points as the keys
                                 and the distance as the value.
            'coo_matrix', 'dok_matrix', 'ndarray'
    """
    
    tree1 = scipy.spatial.cKDTree(points_1, leafsize=16)
    if points_2 is None:
        points_2 = points_1
    tree2 = scipy.spatial.cKDTree(points_2,leafsize=16)
    
    distances = tree1.sparse_distance_matrix(tree2, cutoff, output_type=output)
    return distances



def from_scipy_sparse_matrix(A):
    r"""Converts a scipy sparse matrix to edge indices and edge attributes.

    Args:
        A (scipy.sparse): A sparse matrix.
    """
    A = A.tocoo()
    row = torch.from_numpy(A.row).to(torch.long)
    col = torch.from_numpy(A.col).to(torch.long)
    edge_index = torch.stack([row, col], dim=0)
    edge_weight = torch.from_numpy(A.data)
    return edge_index, edge_weight


def convert_sparse_matrix_to_sparse_tensor(coo):
    values = coo.data
    indices = np.vstack((coo.row, coo.col))

    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = coo.shape
    return torch.sparse.FloatTensor(i, v, torch.Size(shape)).to_dense()

def load_data_adjacency(roi='BRAC4002.3c_ROI3_MRTX.', cutoff= 25, output='dict',label_output='Domain'):

    """
    Input: 
    roi - required reigon of interest: 
    Valid  ROI's are: ['BRAC3495.3f_ROI1_Cont_crop1.', 'BRAC3495.3f_ROI1_Cont_crop2.',
        'BRAC3326.4e_ROI1_Cont_crop1.', 'BRAC3438.6f_ROI1_Cont.',
        'BRAC3438.6f_ROI2_Cont.', 'BRAC3438.6f_ROI3_Cont.', 'BRAC3529.2d_ROI1_MRTX.', 'BRAC4002.3c_ROI2_MRTX_crop1.',
        'BRAC4002.3c_ROI2_MRTX_crop2.', 'BRAC4002.3c_ROI3_MRTX.',
        'BRAC3529.2b_ROI1_MRTX_crop2.', 'BRAC4002.3c_ROI1_MRTX.']
    cutoff - 
    """
    
    full2 = pd.read_csv(path+'20210413_ContMRTX_neighbour_clustering_61clusters_DCsSeparated_TCrenamed_addedInfo copy.csv')
    full2_nandrop=full2.dropna()


    le_domain = preprocessing.LabelEncoder()
    full2_nandrop[label_output] = le_domain.fit_transform(full2_nandrop[label_output])

    loc_all_cell = full2_nandrop[['Location_Center_X','Location_Center_Y']].to_numpy()

    cols_to_drop=['X.1', 'X.2','X.3']

    output_df=full2_nandrop.drop(cols_to_drop, axis=1)


    # data frame for each roi. can write function to do this, or create a dictionary   

    output_df = output_df[(output_df.ROI_name==(roi))]
    output_df = output_df.sample(frac=1).reset_index(drop=True)

    loc_roi = output_df[['Location_Center_X','Location_Center_Y']].to_numpy()
    

    dist_1=distance_matrix(cutoff, loc_roi,loc_roi, 'coo_matrix')

    data , row, col = dist_1.data ,dist_1.row, dist_1.col

    def mini(x):
        if x!=0.0: 
            return 1/x 
        else: return x

    def mini2(x):
        if x!=0.0: 
            return 1
        else: return x

    vfunc = np.vectorize(mini)
    data1 = vfunc(data)
    
    n,_  = loc_roi.shape 
    adj = sp.coo_matrix((data1, (row, col)), shape=(n, n))

    cols_to_drop = ['Location_Center_X','Location_Center_Y']
    output_df=output_df.drop(cols_to_drop,axis=1)


    labels = output_df[label_output].to_numpy()
    features = output_df.drop(label_output,axis=1)

    # indexes_to_encode = [2, 3, 21, 22, 23, 24, 55]

    to_encode = ['cellID', 'cellType', 'Treatment', 'ROI_name', 'Filename', 'Clustername', 'clustername_tumour','Domain']

    if label_output in to_encode: 
        to_encode.remove(label_output)

    for col in to_encode: 
        le_domain = preprocessing.LabelEncoder()
        features[col] = le_domain.fit_transform(features[col])
        
    features=features.to_numpy()
    S=sp.csr_matrix(adj)

    edge_index , edge_weight = from_scipy_sparse_matrix(S)

    print("Data Loaded")

    return S, torch.tensor(features), torch.tensor(labels) , edge_index ,edge_weight



def load_data_adjacency_clusters_with_greater_than_x(roi='BRAC4002.3c_ROI3_MRTX.', cutoff= 25, output='dict',label_output='cluster',cell_cutoff=200):

    """
    Input: 
    roi - required reigon of interest: 
    Valid  ROI's are: ['BRAC3495.3f_ROI1_Cont_crop1.', 'BRAC3495.3f_ROI1_Cont_crop2.',
        'BRAC3326.4e_ROI1_Cont_crop1.', 'BRAC3438.6f_ROI1_Cont.',
        'BRAC3438.6f_ROI2_Cont.', 'BRAC3438.6f_ROI3_Cont.', 'BRAC3529.2d_ROI1_MRTX.', 'BRAC4002.3c_ROI2_MRTX_crop1.',
        'BRAC4002.3c_ROI2_MRTX_crop2.', 'BRAC4002.3c_ROI3_MRTX.',
        'BRAC3529.2b_ROI1_MRTX_crop2.', 'BRAC4002.3c_ROI1_MRTX.']
    cutoff - 
    """
    
    full2 = pd.read_csv(path+'20210413_ContMRTX_neighbour_clustering_61clusters_DCsSeparated_TCrenamed_addedInfo copy.csv')
    full2_nandrop=full2.dropna()


    le_domain = preprocessing.LabelEncoder()
    full2_nandrop[label_output] = le_domain.fit_transform(full2_nandrop[label_output])

    loc_all_cell = full2_nandrop[['Location_Center_X','Location_Center_Y']].to_numpy()

    cols_to_drop=['X.1', 'X.2','X.3']

    t=full2_nandrop['cluster'].value_counts().to_frame()
    valid_clusters=t[t>cell_cutoff].dropna().index
    full2_nandrop=full2_nandrop[full2_nandrop['cluster'].isin(valid_clusters)]

    output_df=full2_nandrop.drop(cols_to_drop, axis=1)



    # data frame for each roi. can write function to do this, or create a dictionary   

    output_df = output_df[(output_df.ROI_name==(roi))]
    output_df = output_df.sample(frac=1).reset_index(drop=True)

    loc_roi = output_df[['Location_Center_X','Location_Center_Y']].to_numpy()
    

    dist_1=distance_matrix(cutoff, loc_roi,loc_roi, 'coo_matrix')

    data , row, col = dist_1.data ,dist_1.row, dist_1.col

    def mini(x):
        if x!=0.0: 
            return 1/x 
        else: return x

    def mini2(x):
        if x!=0.0: 
            return 1
        else: return x

    vfunc = np.vectorize(mini)
    data1 = vfunc(data)
    
    n,_  = loc_roi.shape 
    adj = sp.coo_matrix((data1, (row, col)), shape=(n, n))

    cols_to_drop = ['Location_Center_X','Location_Center_Y']
    output_df=output_df.drop(cols_to_drop,axis=1)


    labels = output_df[label_output].to_numpy()
    features = output_df.drop(label_output,axis=1)

    # indexes_to_encode = [2, 3, 21, 22, 23, 24, 55]

    to_encode = ['cellID', 'cellType', 'Treatment', 'ROI_name', 'Filename', 'Clustername', 'clustername_tumour','Domain']

    if label_output in to_encode: 
        to_encode.remove(label_output)

    for col in to_encode: 
        le_domain = preprocessing.LabelEncoder()
        features[col] = le_domain.fit_transform(features[col])
        
    features=features.to_numpy()

    print(adj.shape,features.shape,labels.shape)

    S=sp.csr_matrix(adj)

    edge_index , edge_weight = from_scipy_sparse_matrix(S)

    print("Data Loaded")

    return S, torch.tensor(features), torch.tensor(labels) , edge_index ,edge_weight


def load_data_adjacency_n_greatest_clusters(roi='BRAC4002.3c_ROI3_MRTX.', cutoff= 25, output='dict',label_output='cluster',n_clusters=20):

    """
    Input: 
    roi - required reigon of interest: 
    Valid  ROI's are: ['BRAC3495.3f_ROI1_Cont_crop1.', 'BRAC3495.3f_ROI1_Cont_crop2.',
        'BRAC3326.4e_ROI1_Cont_crop1.', 'BRAC3438.6f_ROI1_Cont.',
        'BRAC3438.6f_ROI2_Cont.', 'BRAC3438.6f_ROI3_Cont.', 'BRAC3529.2d_ROI1_MRTX.', 'BRAC4002.3c_ROI2_MRTX_crop1.',
        'BRAC4002.3c_ROI2_MRTX_crop2.', 'BRAC4002.3c_ROI3_MRTX.',
        'BRAC3529.2b_ROI1_MRTX_crop2.', 'BRAC4002.3c_ROI1_MRTX.']
    cutoff - 
    """
    
    full2 = pd.read_csv(path+'20210413_ContMRTX_neighbour_clustering_61clusters_DCsSeparated_TCrenamed_addedInfo copy.csv')
    full2_nandrop=full2.dropna()


    le_domain = preprocessing.LabelEncoder()
    full2_nandrop[label_output] = le_domain.fit_transform(full2_nandrop[label_output])

    loc_all_cell = full2_nandrop[['Location_Center_X','Location_Center_Y']].to_numpy()

    cols_to_drop=['X.1', 'X.2','X.3']

    t=full2_nandrop['cluster'].value_counts().to_frame()
    valid_clusters=t.nlargest(n_clusters,'cluster').index
    full2_nandrop=full2_nandrop[full2_nandrop['cluster'].isin( valid_clusters)]

    output_df=full2_nandrop.drop(cols_to_drop, axis=1)



    # data frame for each roi. can write function to do this, or create a dictionary   

    output_df = output_df[(output_df.ROI_name==(roi))]
    output_df = output_df.sample(frac=1).reset_index(drop=True)

    loc_roi = output_df[['Location_Center_X','Location_Center_Y']].to_numpy()
    

    dist_1=distance_matrix(cutoff, loc_roi,loc_roi, 'coo_matrix')

    data , row, col = dist_1.data ,dist_1.row, dist_1.col

    def mini(x):
        if x!=0.0: 
            return 1/x 
        else: return x

    def mini2(x):
        if x!=0.0: 
            return 1
        else: return x

    vfunc = np.vectorize(mini)
    data1 = vfunc(data)
    
    n,_  = loc_roi.shape 
    adj = sp.coo_matrix((data1, (row, col)), shape=(n, n))

    cols_to_drop = ['Location_Center_X','Location_Center_Y']
    output_df=output_df.drop(cols_to_drop,axis=1)


    labels = output_df[label_output].to_numpy()
    features = output_df.drop(label_output,axis=1)

    # indexes_to_encode = [2, 3, 21, 22, 23, 24, 55]

    to_encode = ['cellID', 'cellType', 'Treatment', 'ROI_name', 'Filename', 'Clustername', 'clustername_tumour','Domain']

    if label_output in to_encode: 
        to_encode.remove(label_output)

    for col in to_encode: 
        le_domain = preprocessing.LabelEncoder()
        features[col] = le_domain.fit_transform(features[col])
        
    features=features.to_numpy()

    print(adj.shape,features.shape,labels.shape)

    S=sp.csr_matrix(adj)

    edge_index , edge_weight = from_scipy_sparse_matrix(S)

    print("Data Loaded")

    return S, torch.tensor(features), torch.tensor(labels) , edge_index ,edge_weight


Main model


In [ ]:
import time
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt


dropout=0.5
epochs=50
hidden_dim=64
lr = 0.1
weight_decay=0.0005
t_factor=1
values=[]
plot_accuracy = False
roi = 'BRAC3529.2b_ROI1_MRTX_crop2.'
label_output = 'cluster'

torch.manual_seed(0)

adj, features, labels , edge_index , edge_weight = load_data_adjacency(roi,cutoff=40,label_output=label_output)


n = labels.shape[0]

idx_train = range((int(np.floor(0.7*n)))) 
idx_val = range((int(np.floor(0.7*n))), (int(np.floor(0.8*n))))
idx_test = range((int(np.floor(0.8*n))), (int(np.floor(1*n))))

# print(idx_train,idx_val,idx_test)


model = GCN(features.shape[1],
            hidden_dim,
            labels.max().item() + 1,
            dropout)

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# optimizer = torch.optim.Adam([
#     dict(params=model.gc1.parameters(), weight_decay=weight_decay),
#     dict(params=model.gc2.parameters(), weight_decay=0),
#     dict(params=model.gc3.parameters(), weight_decay=0)
# ], lr=0.01)

print("Training the model")

for epoch in range(epochs):
    t = time.time()
    model.train()
    optimizer.zero_grad()

#     print('features/adj',features,adj)

    output = model(features.float(), edge_index, edge_weight)

    loss_train = F.cross_entropy(output[idx_train], labels[idx_train])
    
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()
    loss_val = F.cross_entropy(output[idx_val], labels[idx_val])

    acc_val = accuracy(output[idx_val], labels[idx_val])
    values.append(acc_train.item())
    print('Epoch: {:04d}'.format(epoch+1),
            'loss_train: {:.4f}'.format(loss_train.item()),
            'acc_train: {:.4f}'.format(acc_train.item()),
            'loss_val: {:.4f}'.format(loss_val.item()),
            'acc_val: {:.4f}'.format(acc_val.item()),
            'time: {:.4f}s'.format(time.time() - t))

"Testing the model"

model.eval()
output = model(features.float(), edge_index, edge_weight)
loss_test = F.nll_loss(output[idx_test], labels[idx_test])
acc_test = accuracy(output[idx_test], labels[idx_test])

print("Test set results:",
        "loss= {:.4f}".format(loss_test.item()),
        "accuracy= {:.4f}".format(acc_test.item()))

if plot_accuracy:
    plt.plot(values)
    plt.xlabel('Epochs')
    plt.ylabel("Accuracy")
    plt.title(f"Epochs: {epochs}, ROI: {roi} , 3 layer GCN: (Relu,Sigmoid,log_softmax), Test Accuracy: {acc_test}, Output: {label_output}")
    plt.savefig(f'plots/20 largest clusters , Epochs: {epochs}, ROI: {roi} , 3 layer GCN: (Relu,sigmoid,log_softmax), Test Accuracy: {acc_test}, Output: {label_output}.png')
    # plt.show()
    